In [ ]:
import os
import subprocess
import sys
import select
import git

In [ ]:
# Directories and Paths (Please use absolute paths)
repos_file_path = ""
autoware_path = ""
scenario_file_path = ""
osm_file_path = ""
pcd_file_path = ""
date_to_stop_searching="" #Please write it in format of "year-month-day" like that "2023-09-31"
os.chdir(autoware_path)
print("The Current working directory now is: {0}".format(os.getcwd()))

In [ ]:
repos_path = []
def get_repos_paths(repos_file_lines):
    sub1 = " "
    sub2 = ":"
    commits = []
    for s in repos_file_lines:
        res = ''
        try:
            idx1 = s.index(sub1)
            idx2 = s.index(sub2)
    
        except ValueError:
            print("Oops!  That was no valid number.  Try again...")
            continue
        for idx in range(idx1 + len(sub1) + 1, idx2):
        #for idx in range(0, idx2):
            res = res + s[idx]
        if ' ' in res or "simulator" in res:
            continue
        repos_path.append(autoware_path+"/src/"+res)
        #print(res)
    return repos_path

In [ ]:
with open(repos_file_path, "r") as file:
    data = file.readlines()
    get_repos_paths(data)
print(repos_path)

In [ ]:
for path in repos_path:
    print(path)

In [ ]:
def split_log_info(repo_log):
    delimiter = "\n\ncommit "
    counter = 0
    splitted_log_info = []
    for x in repo_log.split(delimiter):
        if counter == 0:
            splitted_log_info.append(x)
            counter = counter + 1
        else :
            splitted_log_info.append("commit "+x)
    
    return splitted_log_info

In [ ]:
def get_commits(splitted_log_info):
    sub1 = "commit"
    sub2 = "\nAuthor"
    commits = []
    for s in splitted_log_info:
        res = ''
        try:
            idx1 = s.index(sub1)
            idx2 = s.index(sub2)
    
        except ValueError:
            print("Oops!  That was no valid number.  Try again...")
            continue
        for idx in range(idx1 + len(sub1) + 1, idx2):
            res = res + s[idx]
        commits.append(res)
        #print(res)
    return commits

In [ ]:
repo_commits = []
max_repo_commits_length = 0
repos_commits_dict={}
for repo_path in repos_path:
    #print(repo_path)
    repo_git = git.Git(repo_path)
    #print(repo_git.log(-1))
    repo_log_info = repo_git.log("--since="+date_to_stop_searching)
    if not repo_log_info:
        print(repo_path, "does not have any commits in the specified time period.\nTaking the current commit.")
        repo_log_info = repo_git.log(-1)
    #print(repo_log_info)
    splitted_log = split_log_info(repo_log_info)
    #print(splitted_log)
    #if  splitted_log:
    #print(len(splitted_log))
    repo_commits = get_commits(splitted_log)
    if len(repo_commits) > max_repo_commits_length:
        max_repo_commits_length = len(repo_commits)
    print(repo_commits)
    #else:
     #   print(repo_path, " does not have any commits in this time period")
    print("------/////////////////**************\\\\\\\\\\\-----------")
    repos_commits_dict[repo_path] = repo_commits


print("max_repo_commits_length = " , max_repo_commits_length)

In [ ]:
print(repos_commits_dict)

In [ ]:
counter = 0
for repo in repos_commits_dict.keys():
    counter = counter + 1
    print(repo, "has commit(s):", len(repos_commits_dict[repo]), "and they are :")
    for commit in repos_commits_dict[repo]:
        print(commit)
    print("-----------------------------------------")

print(counter)

In [ ]:
def run_subprocess_with_capture_and_print(cmd):
    p = subprocess.Popen(cmd,
        stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    stdout = []
    stderr = []
    
    while True:
        reads = [p.stdout.fileno(), p.stderr.fileno()]
        ret = select.select(reads, [], [])
    
        for fd in ret[0]:
            if fd == p.stdout.fileno():
                read = p.stdout.readline()
                sys.stdout.write(read.decode())
                stdout.append(read.decode())
            if fd == p.stderr.fileno():
                read = p.stderr.readline()
                sys.stderr.write(read.decode())
                stderr.append(read.decode())
    
        if p.poll() != None:
            break
    
    return stdout, stderr
    

In [ ]:
def compile_autoware():
    cmd = ["colcon","build","--symlink-install","--cmake-args", "-DCMAKE_BUILD_TYPE=Debug", "-DCMAKE_EXPORT_COMPILE_COMMANDS=1"]
    stdout, stderr = run_subprocess_with_capture_and_print(cmd)
    print('Compilate Ended')
    return stdout, stderr
    
    #print('stdout:', "".join(stdout))
    #print('stderr:', "".join(stderr))

In [ ]:
def check_autoware_compile_output(compile_stdout):
    for msg in compile_stdout:
        if "Aborted" in msg or "Failed" in msg:
            return False
    return True

In [ ]:
def run_scenario_simulator():
    with open(scenario_file_path, "r") as file:
        data = file.readlines()
        count = 0
        for line in data:
            if "LogicFile:" in line:
                print("found")
                logic_file = True
                data[count + 1] = "      filepath: "+osm_file_path+"\n"
                logic_file_line_number = count + 1
            if "SceneGraphFile:" in line:
                print("found")
                scene_graphic = True
                data[count + 1] = "      filepath: "+pcd_file_path+"\n"
            count = count + 1
        with open(scenario_file_path, "w") as output:
            output.writelines(data)
    
    cmd = ["ros2","launch","scenario_test_runner","scenario_test_runner.launch.py", "architecture_type:=awf/universe", 
       "record:=false", "scenario:="+scenario_file_path,
       "sensor_model:=sample_sensor_kit", "vehicle_model:=sample_vehicle"]
    
    stdout, stderr = run_subprocess_with_capture_and_print(cmd)
        
    print("Scenario Simulator Run Ended")
    return stdout, stderr

In [ ]:
def check_scenario_simulator_output(sim_stdout):
    number_of_scenarios = 0
    number_of_successful_scenarios = 0
    for msg in sim_stdout:
        if "Shutting down Autoware: (3/3) Waiting for Autoware to be exited" in msg :
            number_of_scenarios = number_of_scenarios + 1
        if "Passed" in msg :
            number_of_successful_scenarios = number_of_successful_scenarios + 1
    if number_of_scenarios > 0:
        if number_of_scenarios == number_of_successful_scenarios:
            return True
    return False

In [ ]:
def print_repos_before_first_success(index):
    for repo in repos_commits_dict.keys():
        commit = "empty"
        if index > len(repos_commits_dict[repo]) - 1 :
            commit = repos_commits_dict[len(repos_commits_dict[repo]) - 1]
        else :
            commit = repos_commits_dict[index]
        print("Repo Name : ", repo, " - Commit ID : ", commit)
        print("--------////-------")

In [ ]:
scenario_pass = False
last_changed_repo = "empty"
last_change_commit = "empty"
index = -1
for i in range(1, max_repo_commits_length):
    for repo in repos_commits_dict.keys():
        if len(repos_commits_dict[repo]) == 1 or i > len(repos_commits_dict[repo])-1:
            continue
        print("Checking out repo", repo, "of commit id ", repos_commits_dict[repo])
        repo_git = git.Git(repo)
        repo_git.checkout(repos_commits_dict[repo][i])
        #compile
        print("Compiling Autoware")
        compile_stdout, compile_stderr = compile_autoware()
        compile_pass = check_autoware_compile_output(compile_stdout)
        #if compile fail .. contine
        if not compile_pass:
            continue
        # run sim
        print("Running Scenario Simulator")
        sim_stdout, sim_stderr = run_scenario_simulator()
        # check if all iterations pass
        sim_pass = check_scenario_simulator_output(sim_stdout)
        if sim_pass:
            print("Found the repos that made the scenario pass with all iterations")
            scenario_pass = True
            last_changed_repo = repo
            last_change_commit = repos_commits_dict[repo]
            index = i - 1
            break
            #print and formate repos file
            # break
    if scenario_pass:
        break
        
        #if true print and break
        #if false continue

In [ ]:
if scenario_pass == False :
    print("The script is not able to find the success/fail border")
else:
    print_repos_before_first_success(index)